## PROTEIN GENERATOR to test runs and play around

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, json

from dataclasses import asdict

import py3Dmol
import torch
import numpy as np
from tqdm import tqdm

from proteome import protein
from proteome.constants.residue_constants import restypes
from proteome.models.design.protein_generator import config
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding
from proteome.models.design.protein_generator.modeling import ProteinGeneratorForJointDesign
from proteome.models.design.protein_generator.sampler import SeqDiffSampler

In [5]:
from proteome.constants.residue_constants import restypes_with_x_dash

In [6]:
"".join(restypes_with_x_dash)

'ARNDCQEGHILKMFPSTWYVX-'

In [4]:
designer = ProteinGeneratorForJointDesign(model_name="default")
folder = OmegaFoldForFolding()

In [6]:
features = designer.design_structure_and_sequence(config.InferenceConfig())

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [8]:
sequence = "".join([restypes[aa] for aa in features["best_seq"].cpu().numpy()])
structure = protein.Protein(
    atom_positions=features["best_xyz"].cpu().numpy()[0],
    aatype=features["best_seq"].cpu().numpy(),
    atom_mask=np.ones((len(features["seq"][0]), 27)),
    residue_index=np.array([t[1] for t in features["pdb_idx"]]),
    b_factors=features["best_pred_lddt"][0].cpu().numpy()[..., None].repeat(27, axis=1),
    chain_index=np.array([protein.PDB_CHAIN_IDS.index(t[0]) for t in features["pdb_idx"]]),
)

In [10]:
designed_pdb = protein.to_pdb(structure)

In [17]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(designed_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
predicted_protein, confidence = folder.fold(sequence)
folded_pdb = protein.to_pdb(predicted_protein)

In [14]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(folded_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol